In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [2]:
with open('../data/best_lr_retrained_tfidf.pckl', 'rb') as f:
    lr = pickle.load(f)

with open('../data/best_svm_retrained_tfidf.pckl', 'rb') as f:
    svm = pickle.load(f)

with open('../data/best_mnb_retrained_tfidf.pckl', 'rb') as f:
    mnb = pickle.load(f)

In [3]:
with open(f'../data/test_tfidf_subsampled_data_3.pckl', 'rb') as f:
    test_full = pickle.load(f)

with open(f'../data/test_tfidf_subsampled_data_sliced_2.pckl', 'rb') as f:
    test_sliced = pickle.load(f)

In [4]:
test_full.shape, test_sliced.shape

((72165, 4047745), (72165, 98489))

In [5]:
predicted_lr = lr.predict_proba(test_sliced)[:, 1]
predicted_svm = svm.predict_proba(test_full)[:, 1]
predicted_mnb = mnb.predict_proba(test_full)[:, 1]

with open('predicted_test_labels.pckl', 'wb') as f:
    pickle.dump((predicted_lr,
                 predicted_svm,
                 predicted_mnb),
                f)

In [6]:
predicted_lr[:10]

array([0.1580181 , 0.32280205, 0.45910523, 0.44793096, 0.33488793,
       0.45373827, 0.61693852, 0.27379469, 0.5152519 , 0.32494541])

In [7]:
predicted_svm[:10]

array([0.25657769, 0.36497999, 0.54282186, 0.61058831, 0.44647988,
       0.3269297 , 0.64856411, 0.33657698, 0.47931043, 0.388763  ])

In [8]:
predicted_mnb[:10]

array([4.35291490e-02, 1.83143485e-01, 8.60648832e-02, 5.69041774e-01,
       5.88446288e-01, 5.53086361e-04, 5.46054150e-01, 3.43760906e-02,
       1.01969620e-01, 2.25691819e-01])

In [9]:
(predicted_lr > 0.5).sum(), (predicted_svm > 0.5).sum(), (predicted_mnb > 0.5).sum()

(16290, 17689, 7404)

In [10]:
ensemble_df = pd.DataFrame([predicted_lr, predicted_svm, predicted_mnb]).T
ensemble_df.columns = ['lr', 'svm', 'mnb']
ensemble_df['vote'] = ((ensemble_df[['lr', 'svm', 'mnb']] > 0.5).sum(axis=1) > 2).astype(int)
ensemble_df['vote_proba'] = (ensemble_df[['lr', 'svm', 'mnb']] > 0.5).sum(axis=1) / 3
ensemble_df['avg'] = ensemble_df[['lr', 'svm', 'mnb']].mean(axis=1)
ensemble_df.head()

,lr,svm,mnb,vote,vote_proba,avg
0,0.158018,0.256578,0.043529,0,0.000000,0.152708
1,0.322802,0.364980,0.183143,0,0.000000,0.290309
2,0.459105,0.542822,0.086065,0,0.333333,0.362664
3,0.447931,0.610588,0.569042,0,0.666667,0.542520
4,0.334888,0.446480,0.588446,0,0.333333,0.456605


In [11]:
ensemble_df['avg'].to_csv('../data/predictions.csv', index=False, header=False)

In [12]:
ensemble_df.shape

(72165, 6)